In [1]:
# Importing and data
import theano.tensor as T
import theano
import sys, os
sys.path.append("../GeMpy")

# Importing GeMpy modules
import GeMpy

# Reloading (only for development purposes)
import importlib
importlib.reload(GeMpy)

# Usuful packages
import numpy as np
import pandas as pn

import matplotlib.pyplot as plt

# This was to choose the gpu
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

# Default options of printin
np.set_printoptions(precision = 6, linewidth= 130, suppress =  True)

#%matplotlib inline
%matplotlib inline



# Setting the extent
geo_data = GeMpy.import_data([0,10,0,10,0,10], [50,50,50])


# =========================
# DATA GENERATION IN PYTHON
# =========================
# Layers coordinates
layer_1 = np.array([[0.5,4,7], [2,4,6.5], [4,4,7], [5,4,6]])#-np.array([5,5,4]))/8+0.5
layer_2 = np.array([[3,4,5], [6,4,4],[8,4,4], [7,4,3], [1,4,6]])
layers = np.asarray([layer_1,layer_2])

# Foliations coordinates
dip_pos_1 = np.array([7,4,7])#- np.array([5,5,4]))/8+0.5
dip_pos_2 = np.array([2.,4,4])

# Dips
dip_angle_1 = float(15)
dip_angle_2 = float(340)
dips_angles = np.asarray([dip_angle_1, dip_angle_2], dtype="float64")

# Azimuths
azimuths = np.asarray([90,90], dtype="float64")

# Polarity
polarity = np.asarray([1,1], dtype="float64")

# Setting foliations and interfaces values
GeMpy.set_interfaces(geo_data, pn.DataFrame(
    data = {"X" :np.append(layer_1[:, 0],layer_2[:,0]),
            "Y" :np.append(layer_1[:, 1],layer_2[:,1]),
            "Z" :np.append(layer_1[:, 2],layer_2[:,2]),
            "formation" : np.append(
               np.tile("Layer 1", len(layer_1)), 
               np.tile("Layer 2", len(layer_2))),
            "labels" : [r'${\bf{x}}_{\alpha \, 0}^1$',
               r'${\bf{x}}_{\alpha \, 1}^1$',
               r'${\bf{x}}_{\alpha \, 2}^1$',
               r'${\bf{x}}_{\alpha \, 3}^1$',
               r'${\bf{x}}_{\alpha \, 0}^2$',
               r'${\bf{x}}_{\alpha \, 1}^2$',
               r'${\bf{x}}_{\alpha \, 2}^2$',
               r'${\bf{x}}_{\alpha \, 3}^2$',
        
                        r'${\bf{x}}_{\alpha \, 4}^2$'] }))

GeMpy.set_foliations(geo_data,  pn.DataFrame(
    data = {"X" :np.append(dip_pos_1[0],dip_pos_2[0]),
            "Y" :np.append(dip_pos_1[ 1],dip_pos_2[1]),
            "Z" :np.append(dip_pos_1[ 2],dip_pos_2[2]),
            "azimuth" : azimuths,
            "dip" : dips_angles,
            "polarity" : polarity,
            "formation" : ["Layer 1", "Layer 2"],
            "labels" : [r'${\bf{x}}_{\beta \,{0}}$',
              r'${\bf{x}}_{\beta \,{1}}$'] })) 

In [7]:
renderer.SetBackground?

In [90]:
import vtk
 
# Call back function
def sphereCallback(obj, event):
    camera.SetFocalPoint(5, 5, 5);
    camera.SetPosition(0,5,5)
    camera.SetViewUp(0,0,1)
    renderer.ResetCamera()
    print('Center: {}, {}, {}'.format(*obj.GetCenter()))
 
# A renderer and render window
renderer = vtk.vtkRenderer()
# renderer.SetBackground(10, 10, 10)
#renderer.SetBackground(0, 0, 1)
renwin = vtk.vtkRenderWindow()
renwin.AddRenderer(renderer)
renwin.SetSize(700, 700)
renwin.SetWindowName('RW: Multiple ViewPorts')
#renderer.SetViewport(0.8,0,1,1)

# An interactor
interactor = vtk.vtkRenderWindowInteractor()
interactor.SetRenderWindow(renwin)




# A Sphere widget
sphereWidget = vtk.vtkSphereWidget()
sphereWidget.SetCenter(5,5,1)
sphereWidget.SetInteractor(interactor)
sphereWidget.SetRepresentationToSurface()
sphereWidget.On()

sphereWidget2 = vtk.vtkSphereWidget()
sphereWidget2.SetInteractor(interactor)
sphereWidget2.SetRepresentationToSurface()
sphereWidget2.On()
 
# Connect the event to a function
sphereWidget.AddObserver("InteractionEvent", sphereCallback)
sphereWidget2.AddObserver("InteractionEvent", sphereCallback)
 
    
    


cubeAxesActor = vtk.vtkCubeAxesActor()
cubeAxesActor.SetBounds(0, 10, 0, 10, 0, 10)
#cubeAxesActor.SetAxisBaseForZ(0,1,0)
#cubeAxesActor.SetAxisBaseForY(0,0,1)
cubeAxesActor.SetCamera(renderer.GetActiveCamera())
cubeAxesActor.GetTitleTextProperty(0).SetColor(1.0, 0.0, 0.0)
cubeAxesActor.GetLabelTextProperty(0).SetColor(1.0, 0.0, 0.0)
 
cubeAxesActor.GetTitleTextProperty(1).SetColor(0.0, 1.0, 0.0)
cubeAxesActor.GetLabelTextProperty(1).SetColor(0.0, 1.0, 0.0)
 
cubeAxesActor.GetTitleTextProperty(2).SetColor(1.0, 1.0, 1.0)
cubeAxesActor.GetLabelTextProperty(2).SetColor(1.0, 1.0, 1.0)
 
cubeAxesActor.DrawXGridlinesOn()
cubeAxesActor.DrawYGridlinesOn()
cubeAxesActor.DrawZGridlinesOn()
if vtk.VTK_MAJOR_VERSION > 5:
    cubeAxesActor.SetGridLineLocation(vtk.VTK_GRID_LINES_FURTHEST)
 
# cubeAxesActor.XAxisMinorTickVisibilityOff()
# cubeAxesActor.YAxisMinorTickVisibilityOff()
# cubeAxesActor.ZAxisMinorTickVisibilityOff()
 
renderer.AddActor(cubeAxesActor)

camera =vtk.vtkCamera ();
camera.SetFocalPoint(5, 5, 5);
camera.SetPosition(0,5,5)
camera.SetViewUp(0,0,1)


renderer.SetActiveCamera(camera);

renderer.ResetCamera()
cubeAxesActor.SetCamera(renderer.GetActiveCamera())


# AXES
transform = vtk.vtkTransform()
transform.Translate(1.0, 0.0, 0.0)

 
# properties of the axes labels can be set as follows
# this sets the x axis label to red
# axes->GetXAxisCaptionActor2D()->GetCaptionTextProperty()->SetColor(1,0,0);
 
# the actual text of the axis label can be changed:
# axes->SetXAxisLabelText("test");
 
#renderer.AddActor(axes)

arrowSource = vtk.vtkArrowSource()
#arrowSource.SetShaftRadius(0.01)
#arrowSource.SetTipLength(.9)
 
# Create a mapper and actor
mapper = vtk.vtkPolyDataMapper()
mapper.SetInputConnection(arrowSource.GetOutputPort())
actor = vtk.vtkActor()
actor.SetMapper(mapper)
actor.AddObserver("InteractionEvent", sphereCallback)
renderer.AddActor(actor)

# Start
interactor.Initialize()
interactor.Start()

del renwin, interactor

Center: 5.0, 5.0, 1.0
Center: 5.0, 5.0, 1.0
Center: 5.0, 5.0, 1.0
Center: 5.0, 5.0, 1.0
Center: 5.0, 5.0, 1.0
Center: 5.0, 5.0, 1.0
Center: 5.0, 5.0, 1.0
Center: 5.0, 5.0, 1.0
Center: 5.0, 5.0, 1.0
Center: 5.0, 5.0, 1.0
Center: 5.0, 5.0, 1.0
Center: 0.0, 0.026246149905908234, 0.0
Center: 0.0, 0.05246991244410837, 0.0
Center: 0.0, 0.05246991244410837, 0.02620143259110852
Center: 0.0, 0.07864894388114102, 0.02620143259110852
Center: 0.0, 0.10480570271692219, 0.02620143259110852
Center: 0.0, 0.10480570271692219, 0.052335976108302484
Center: 0.0, 0.13091790233202016, 0.052335976108302484
Center: 0.0, 0.1570079438156739, 0.052335976108302484
Center: 0.0, 0.20916630747303933, 0.052335976108302484
Center: 0.0, 0.23524548930172107, 0.07841515793698611
Center: 0.0, 0.2613023843709433, 0.07841515793698611
Center: 0.0, 0.28735927944016926, 0.07841515793698611
Center: 0.0, 0.3134161745093933, 0.07841515793698611
Center: 0.0, 0.3394730695786193, 0.07841515793698611
Center: 0.0, 0.3655299646478434, 

In [87]:
axes.SetPosition(-5, -5, - 5)

In [25]:
camera =vtk.vtkCamera ();
camera.

In [37]:
interactor = vtk.vtkRenderWindowInteractor()
interactor.SetEventInformation()

TypeError: no overloads of SetEventInformation() take 0 arguments

In [23]:
renderer = vtk.vtkRenderer()
# renderer.SetBackground(10, 10, 10)
 
renderer.SetViewport?

### Box limit?

In [67]:
import vtk
 
# Call back function to resize the cone
def boxCallback(obj, event):
    t = vtk.vtkTransform()
    obj.GetTransform(t)
    obj.GetProp3D().SetUserTransform( t )

    
    
    
    
# A renderer and render window
renderer = vtk.vtkRenderer()
renderer.SetBackground(0, 0, 1)
# renderer.AddActor(coneActor)
 
renwin = vtk.vtkRenderWindow()
renwin.AddRenderer(renderer)
 
# An interactor
interactor = vtk.vtkRenderWindowInteractor()
interactor.SetRenderWindow(renwin)



cubeAxesActor = vtk.vtkCubeAxesActor()
cubeAxesActor.SetBounds(0, 10, 0, 10, 0, 10)
cubeAxesActor.SetCamera(renderer.GetActiveCamera())
cubeAxesActor.GetTitleTextProperty(0).SetColor(1.0, 0.0, 0.0)
cubeAxesActor.GetLabelTextProperty(0).SetColor(1.0, 0.0, 0.0)
 
cubeAxesActor.GetTitleTextProperty(1).SetColor(0.0, 1.0, 0.0)
cubeAxesActor.GetLabelTextProperty(1).SetColor(0.0, 1.0, 0.0)
 
cubeAxesActor.GetTitleTextProperty(2).SetColor(0.0, 0.0, 1.0)
cubeAxesActor.GetLabelTextProperty(2).SetColor(0.0, 0.0, 1.0)
 
#cubeAxesActor.DrawXGridlinesOn()
#cubeAxesActor.DrawYGridlinesOn()
#cubeAxesActor.DrawZGridlinesOn()
if vtk.VTK_MAJOR_VERSION > 5:
    cubeAxesActor.SetGridLineLocation(vtk.VTK_GRID_LINES_FURTHEST)
 
cubeAxesActor.XAxisMinorTickVisibilityOff()
cubeAxesActor.YAxisMinorTickVisibilityOff()
cubeAxesActor.ZAxisMinorTickVisibilityOff()

  
renderer.AddActor(cubeAxesActor)

# A Box widget
boxWidget = vtk.vtkBoxWidget()
boxWidget.SetInteractor(interactor)
boxWidget.SetProp3D(cubeAxesActor)
boxWidget.SetPlaceFactor( 1.25 ) # Make the box 1.25x larger than the actor
boxWidget.PlaceWidget()
boxWidget.On()
 
# Connect the event to a function
boxWidget.AddObserver("InteractionEvent", boxCallback)
 
# Start
interactor.Initialize()
interactor.Start()

del renwin, interactor

In [58]:
cubeAxesActor.GetBounds()

(-1.0, 1.0, -1.0, 1.0, -1.0, 1.0)

In [40]:
#!/usr/bin/env python
 
import vtk
 
# Create a superquadric
superquadricSource = vtk.vtkSuperquadricSource()
superquadricSource.SetPhiRoundness(3.1)
superquadricSource.SetThetaRoundness(1.0)
superquadricSource.Update()  # needed to GetBounds later
 
renderer = vtk.vtkRenderer()
 
mapper = vtk.vtkPolyDataMapper()
mapper.SetInputConnection(superquadricSource.GetOutputPort())
 
superquadricActor = vtk.vtkActor()
superquadricActor.SetMapper(mapper)
 
cubeAxesActor = vtk.vtkCubeAxesActor()
cubeAxesActor.SetBounds(superquadricSource.GetOutput().GetBounds())
cubeAxesActor.SetCamera(renderer.GetActiveCamera())
cubeAxesActor.GetTitleTextProperty(0).SetColor(1.0, 0.0, 0.0)
cubeAxesActor.GetLabelTextProperty(0).SetColor(1.0, 0.0, 0.0)
 
cubeAxesActor.GetTitleTextProperty(1).SetColor(0.0, 1.0, 0.0)
cubeAxesActor.GetLabelTextProperty(1).SetColor(0.0, 1.0, 0.0)
 
cubeAxesActor.GetTitleTextProperty(2).SetColor(0.0, 0.0, 1.0)
cubeAxesActor.GetLabelTextProperty(2).SetColor(0.0, 0.0, 1.0)
 
cubeAxesActor.DrawXGridlinesOn()
cubeAxesActor.DrawYGridlinesOn()
cubeAxesActor.DrawZGridlinesOn()
if vtk.VTK_MAJOR_VERSION > 5:
    cubeAxesActor.SetGridLineLocation(vtk.VTK_GRID_LINES_FURTHEST)
 
cubeAxesActor.XAxisMinorTickVisibilityOff()
cubeAxesActor.YAxisMinorTickVisibilityOff()
cubeAxesActor.ZAxisMinorTickVisibilityOff()
 
renderer.AddActor(cubeAxesActor)
renderer.AddActor(superquadricActor)
renderer.GetActiveCamera().Azimuth(30)
renderer.GetActiveCamera().Elevation(30)
 
renderer.ResetCamera()
 
renderWindow = vtk.vtkRenderWindow()
renderWindow.AddRenderer(renderer)
 
renderWindowInteractor = vtk.vtkRenderWindowInteractor()
renderWindowInteractor.SetRenderWindow(renderWindow)
 
renderWindow.Render()
renderWindowInteractor.Start()

del renderWindow, renderWindowInteractor

In [31]:
cubeAxesActor.SetBounds?